In [98]:
from math import log
import random

In [99]:
class Value:
    def __init__(self, value, op=None, parents=()):
        self.data = value
        self.grad = 0.0
        self.parents = parents
        self.op = op
        self._backward = lambda: None

    def __add__(self, other):
        out = Value(self.data + other.data, "+", (self, other))

        def _backward():
            self.grad += out.grad
            other.grad += out.grad

        out._backward = _backward       
        return out

    def __mul__(self, other):
        out = Value(self.data * other.data, "*", (self, other))

        def _backward(): 
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad

        out._backward = _backward
        return out

    def __sub__(self, other):
        out = Value(self.data - other.data, "-", (self, other))

        def _backward():
            self.grad += out.grad
            other.grad -= out.grad

        out._backward = _backward
        return out
    
    def __pow__(self, other):
        out = Value(self.data ** other.data, "**", (self, other))

        def _backward():
            self.grad += other.data * (self.data ** ( other.data - 1 )) * out.grad
            if self.data > 0:
                other.grad += (self.data ** other.data) * log(self.data) * out.grad
        out._backward = _backward
        return out
    
    def relu(self):
        out = Value(self.data if self.data > 0 else 0, "relu", (self,))

        def _backward():
            self.grad += out.grad if self.data > 0 else 0
        
        out._backward = _backward
        return out

    def __repr__(self):
        return f"Value(data={self.data})"
    
    def backward(self):
        # Topological order all nodes
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for parent in v.parents:
                    build_topo(parent)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for v in reversed(topo):
            v._backward()

In [199]:
class Neuron:
    def __init__(self, x_dim):
        self.weights = [Value(random.uniform(-1, 1)) for _ in range(x_dim)]
        self.bias = Value(random.uniform(-1, 1))
        self.out = None
        self.parameters = self.weights + [self.bias]

    def __call__(self, x):
        self.out = sum((w * x for w, x in zip(self.weights, x)), self.bias)
        self.out = self.out.relu()
        return self.out
    
    def backward(self):
        self.out.backward()

    def zero_grad(self):
        for param in self.parameters:
            param.grad = 0.0

    def update(self):
        for param in self.parameters:
            param.data = 0.01 * param.grad

In [200]:
x = [Value(1.0), Value(3.0), Value(2.0)]
n = Neuron(3)
n(x)

Value(data=3.9131338932043183)

In [204]:
n(x)

Value(data=3.9131338932043183)

In [205]:
n.backward()

In [206]:
print(n.parameters)

[Value(data=0.25090368136450825), Value(data=0.6541461578022143), Value(data=0.6501896372675686), Value(data=0.39941246389802987)]
